In [ ]:
import os
import numpy as np
import pandas as pd
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer().fit(train.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mb.transform(train.labels.apply(lambda x : x.split())), columns = mb.classes_)
new_train = pd.concat([train['image'], labels], axis=1)
new_train.head()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
pre = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range = 0.2,
    shear_range=0.3,
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split= 0.2,)
pos = ImageDataGenerator(rescale = 1./255)

In [ ]:
seed = 88
target_size = (299, 299)
batch_size = 32
resize_img = '../input/resized-plant2021/img_sz_384'
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

# colab
# resize_img = '/content/drive/MyDrive/FGVC8/python工研院課程/專題/data/resize/img_sz_384'
# test_img = '/content/drive/MyDrive/FGVC8/python工研院課程/專題/data/test_images'
# submission = pd.read_csv('/content/drive/MyDrive/FGVC8/python工研院課程/專題/data/sample_submission.csv')

submission.head()

In [ ]:
train_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = resize_img, 
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "training",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
valid_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = resize_img,
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "validation",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
test_generator = pos.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = None,
                  batch_size=16,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed,)

In [ ]:
import matplotlib.pyplot as plt
example = next(train_generator)
print(example[0].shape)
plt.imshow(example[0][0,:,:,:])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
example = next(test_generator)
print(example[0].shape)
plt.imshow(example[0])
plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.applications import InceptionV3
# from tensorflow.keras import layers, optimizers
# from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
# from keras import Model
# base_model = InceptionV3(include_top=False, 
#                 weights='imagenet', 
#                 pooling='avg', 
#                 input_shape=(299,299,3))

In [ ]:
# model_pre = tf.keras.Sequential([
#         base_model,
#         layers.BatchNormalization(),
#         layers.Dense(32,activation='relu'),
#         layers.Dropout(0.4),
#         layers.Dense(16,activation='relu'),
#         layers.Dropout(0.4),
#         layers.Dense(6,activation='sigmoid')
# ])

In [ ]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# model_pre.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#     loss='binary_crossentropy',
#     metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),tf.keras.metrics.AUC(name='auc')])
# model_checkpoint = ModelCheckpoint(
#             filepath="/content/drive/MyDrive/FGVC8/python工研院課程/專題/data/0718_299_incv3.h5", 
#             mode='max',
#             monitor='val_auc', 
#             save_best_only=True, 
#             save_freq="epoch", 
#             verbose=1)
# early_stopping = EarlyStopping(
#             monitor='val_loss',
#             mode='min', 
#             min_delta=0.003,
#             patience=10, 
#             verbose=1, 
#             restore_best_weights=True)

# rlr = keras.callbacks.ReduceLROnPlateau(
#             monitor='val_auc', 
#             mode='auto',
#             factor=0.2, 
#             patience=2, 
#             verbose=1,
#             min_delta=0.0001)

# model_pre = model_pre.fit(train_generator,
#             validation_data=valid_generator,
#             steps_per_epoch=(train_generator.samples//batch_size),
#             validation_steps=(valid_generator.samples//batch_size),
#             epochs=30,
#             callbacks=[model_checkpoint, early_stopping])

In [ ]:
# import matplotlib.pyplot as plt

# pre_history = model_pre.history

# plt.figure()
# plt.plot(pre_history['binary_accuracy'])
# plt.plot(pre_history['val_binary_accuracy'])
# plt.title('model binary_accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('accuracy')
# plt.show()

In [ ]:
# pre_history = model_pre.history

# plt.figure()
# plt.plot(pre_history['auc'])
# plt.plot(pre_history['val_auc'])
# plt.title('model auc')
# plt.ylabel('auc')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('auc')
# plt.show()

In [ ]:
#
import keras
def load_model():
    model_pre = keras.models.load_model("../input/0718-299-invc3/0718_299_incv3.h5")
    return model_pre
model = load_model()

In [ ]:
# base_model.trainable = True

# test_generator = pos.flow_from_dataframe(
#                   submission,
#                   directory = test_img,
#                   x_col = 'image',
#                   y_col = 'labels',
#                   class_mode = None,
#                   batch_size=4,
#                   target_size = target_size,
#                   color_mode="rgb",
#                   shuffle = False,
#                   seed = seed,)

# model = model.fit(
#     train_generator,
#     validation_data=valid_generator,
#     steps_per_epoch=(train_generator.samples//batch_size),
#     validation_steps=(valid_generator.samples//batch_size),
#     epochs=30,
#     callbacks=[model_checkpoint, early_stopping])

In [ ]:
# import matplotlib.pyplot as plt
# model_history = model.history

# plt.figure()
# plt.plot(model_history['binary_accuracy'])
# plt.plot(model_history['val_binary_accuracy'])
# plt.title('model binary_accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('accuracy')
# plt.show()

In [ ]:
#
model = load_model()
pred = model.predict(test_generator)

In [ ]:
perdict = (pred>0.30)
n_label = new_train.columns.tolist()[1:]
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
np.around(pred, decimals=3, out=None)

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)